# Bitcoin price visualiser

In [ ]:
import os
import quandl
import pickle
import plotly
import numpy as pd
import pandas as pd

import config

import plotly.offline as py # offline version of plotly 
import plotly.graph_objs as go  # plotly graphical object
py.init_notebook_mode(connected=True)

In [ ]:
quandl.ApiConfig.api_key = config.key

## Cache bitcoin data

In [ ]:
def get_quandl_data(quandl_code):
    """
    Provide price data for given quandl_code via caching mechanism 
    """
    cache_path = "{}.pkl".format(quandl_code).replace("/", "-")
    
    # load price data if it exists
    try:
        file = open(cache_path, "rb")
        df = pickle.load(file)
        print(f"Loaded {quandl_code} data from cache")
    # othewise cache the data
    except (OSError, IOError):
        df = quandl.get(quandl_code, returns="pandas")
        df.to_pickle(cache_path)
        print(f"Cached {quandl_code} data to {cache_path}")
    return df

In [ ]:
btc_usd_kraken = get_quandl_data("BCHARTS/KRAKENUSD")

In [ ]:
btc_usd_kraken.head()

In [ ]:
# setting up traces for plotting
btc_trace = go.Scatter(x=btc_usd_kraken.index, y=btc_usd_kraken["Weighted Price"])
btc_trace

Trace has been setup correctly so we can proceed to plotting the data

In [ ]:
py.iplot([btc_trace])

The plot shows that there are days where the bitcoin price data is zero even though this has not occurred in the actual price history. Hence, data from other exchanges will need to be aggregated for more realistic price history.

In [ ]:
btc_usd_kraken = get_quandl_data("BCHARTS/KRAKENUSD")

## Get data from multiple exchanges

In [ ]:
# popular exchanges
exchanges = ["COINBASE", "OKCOIN", "BITSTAMP", "KRAKEN", "BITFINEX", "ITBIT"]

# collate exchange data
exchange_data = {}

for exchange in exchanges:
    exchange_code = f"BCHARTS/{exchange}USD"
    exchange_data[exchange] = get_quandl_data(exchange_code)

In [ ]:
def merge_datafs(dataframes, labels, col):
    """
    Make dataframe which includes specified data from each exchanges dataframe
    """
    series_dict = {}
    
    # extract given col from each exchange dataframe
    for index in range(len(labels)):
        series_dict[labels[index]] = dataframes[index][col]
    return pd.DataFrame(series_dict)

In [ ]:
btc_usd_df = merge_datafs(list(exchange_data.values()), list(exchange_data.keys()), "Weighted Price")

Most exchanges don't have data for the genesis of bitcoin

In [ ]:
btc_usd_df.head()

But we do have more receent information so we'll take the average of the prices from each exchange which has prices

In [ ]:
btc_usd_df.tail()

## Visualise aggregate btc price data

In [ ]:
layout = go.Layout(
        title="Bitcoin price by exchange",
        legend={"orientation":"h"},
        xaxis={"type":"date"},
        yaxis={"title":"price ($)"})

# setting up traces for plotting
trace_arr = []
labels = list(btc_usd_df)

for _, label in enumerate(labels):
    series = btc_usd_df[label]
    trace = go.Scatter(x=series.index, y=series, name=label)
    trace_arr.append(trace)
    
fig = go.Figure(data=trace_arr, layout=layout)
py.iplot(fig)

Average the data to address price discontinuities across exchanges

In [ ]:
btc_usd_df["Average Weighted Price"] = btc_usd_df.mean(axis=1)

In [ ]:
btc_mean_trace = go.Scatter(x=btc_usd_df.index, 
                            y=btc_usd_df["Average Weighted Price"], name="Average Price")
py.iplot([btc_mean_trace])